In [19]:
import os
import re
import requests
from io import StringIO 

import gspread
import numpy as np
import pandas as pd
# Tokenizer 
from transformers import T5Tokenizer
# PyTorch model 
from transformers import T5Model, T5ForConditionalGeneration

In [2]:
sheet_url = "https://docs.google.com/spreadsheet/ccc?key=1H7l1w0qORvyRsF07TvK8g4Bz63tdLwpqLOaXhVkw9lQ&output=csv"
r = requests.get(sheet_url)
data = r.content
decoded_data = data.decode()

In [3]:
df = pd.read_csv(StringIO(decoded_data), index_col=1)
df.head()

,published_date,url,file_name,responsible,snippet_1,snippet_2,snippet_3,snippet_4,snippet_5,length
published_title,,,,,,,,,,
187-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/73/E7/C5/27/2...,2023-11-09|187-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 13/11/2023, inclusive, será altera...",A regra de cadastro automático de vencimentos ...,os demais termos do Contrato Futuro de Cupom d...,A regra vigente de cadastro automático e cadas...,Esclarecimentos adicionais poderão ser obtidos...,487.0
107-2023-VNC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/78/67/00/8C/C...,2023-11-09|107-2023-VNC-Comunicado Externo.pdf,Gabriel,a data para a entrada do serviço em ambiente d...,"em 11/11/2023 entre 10h30 e 12h30, haverá exce...","possibilidade de realizar, no dia supracitado,...",Esclarecimentos adicionais poderão ser obtidos...,NaN,393.0
058-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/43/00/01/01/9...,2023-11-09|058-2023-VPC-Comunicado Externo.pdf,Gabriel,"em 27/10/2023, a Receita Federal do Brasil (RF...",prevê a obrigatoriedade do envio das informaçõ...,A partir do aprendizado gerado pelas fases ant...,NaN,NaN,402.0
059-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/31/60/A7/FF/A...,2023-11-09|059-2023-VPC-Comunicado Externo.pdf,Gabriel,"A B3 informa que, no dia 13/11/2023, atualizar...",Registro emissor CVM (Sim ou Não); e Fase (Ope...,A marcação de EGEM ou EFRF será retirada,A atualização em referência será realizada par...,Esclarecimentos adicionais poderão ser obtidos...,501.0
186-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/0F/44/94/03/7...,2023-11-09|186-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 27/11/2023, serão implementados no...",Quantidade média negociada,Quantidade em relação ao capital social do emi...,Negociabilidade,Padronização de leilões de opções,269.0


In [4]:
data_df = df[
    (df["length"] != 0) &
    (df["length"].notnull())
    ]


In [5]:
data_df

,published_date,url,file_name,responsible,snippet_1,snippet_2,snippet_3,snippet_4,snippet_5,length
published_title,,,,,,,,,,
187-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/73/E7/C5/27/2...,2023-11-09|187-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 13/11/2023, inclusive, será altera...",A regra de cadastro automático de vencimentos ...,os demais termos do Contrato Futuro de Cupom d...,A regra vigente de cadastro automático e cadas...,Esclarecimentos adicionais poderão ser obtidos...,487.0
107-2023-VNC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/78/67/00/8C/C...,2023-11-09|107-2023-VNC-Comunicado Externo.pdf,Gabriel,a data para a entrada do serviço em ambiente d...,"em 11/11/2023 entre 10h30 e 12h30, haverá exce...","possibilidade de realizar, no dia supracitado,...",Esclarecimentos adicionais poderão ser obtidos...,NaN,393.0
058-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/43/00/01/01/9...,2023-11-09|058-2023-VPC-Comunicado Externo.pdf,Gabriel,"em 27/10/2023, a Receita Federal do Brasil (RF...",prevê a obrigatoriedade do envio das informaçõ...,A partir do aprendizado gerado pelas fases ant...,NaN,NaN,402.0
059-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/31/60/A7/FF/A...,2023-11-09|059-2023-VPC-Comunicado Externo.pdf,Gabriel,"A B3 informa que, no dia 13/11/2023, atualizar...",Registro emissor CVM (Sim ou Não); e Fase (Ope...,A marcação de EGEM ou EFRF será retirada,A atualização em referência será realizada par...,Esclarecimentos adicionais poderão ser obtidos...,501.0
186-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/0F/44/94/03/7...,2023-11-09|186-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 27/11/2023, serão implementados no...",Quantidade média negociada,Quantidade em relação ao capital social do emi...,Negociabilidade,Padronização de leilões de opções,269.0


In [6]:
save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/"

In [7]:
docs_cols = [
    "published_date", 
    "published_title", 
    "published_abstract", 
    "published_subject", 
    "url",
    "file_name",
    "pypdfium_extraction"
]
docs_df = pd.read_parquet(f"{save_path}list_b3_pdf_comparison.parquet", engine="pyarrow")
docs_df = docs_df[docs_cols]

docs_df = docs_df[docs_df["published_date"].dt.year == 2023]

In [8]:
docs_df.head()

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdfium_extraction
0,2023-11-09,187-2023-PRE-Ofício Circular,"Informamos que, a partir de 13/11/2023, inclus...",Alteração na Regra de Cadastro Automático de V...,https://www.b3.com.br/data/files/73/E7/C5/27/2...,2023-11-09|187-2023-PRE-Ofício Circular.pdf,Este documento produz efeitos a partir da data...
1,2023-11-09,107-2023-VNC-Comunicado Externo,"A B3 informa que, conforme Comunicado Externo ...",Lançamento das sessões de Renda Fixa Trademate,https://www.b3.com.br/data/files/78/67/00/8C/C...,2023-11-09|107-2023-VNC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...
2,2023-11-09,058-2023-VPC-Comunicado Externo,"Informamos que, em 27/10/2023, a Receita Feder...",Imposto de Renda da Pessoa Física (IRPF) – Cál...,https://www.b3.com.br/data/files/43/00/01/01/9...,2023-11-09|058-2023-VPC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...
3,2023-11-09,059-2023-VPC-Comunicado Externo,"A B3 informa que, os dados indicados neste Com...",Atualização cadastral de companhias emissoras ...,https://www.b3.com.br/data/files/31/60/A7/FF/A...,2023-11-09|059-2023-VPC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...
4,2023-11-09,186-2023-PRE-Ofício Circular,"Informamos que, a partir de 27/11/2023, serão ...",Novas Regras e Parâmetros para Procedimentos E...,https://www.b3.com.br/data/files/0F/44/94/03/7...,2023-11-09|186-2023-PRE-Ofício Circular.pdf,Este documento produz efeitos a partir da data...


In [9]:
pattern_1 = r"\r\n|\n|\r"
pattern_2 = r"Este documento.*?Fax.*?PUBLIC INFORMATION "

docs_df["cleaned_extraction"] = (
    docs_df["pypdfium_extraction"]
    .str.replace(pattern_1, " ", regex=True) # Remove line breaks
    .str.replace(pattern_2, "", regex=True) # Clean Header and Footer
)

docs_df.index = docs_df["published_title"]

docs_df.head()

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdfium_extraction,cleaned_extraction
published_title,,,,,,,,
187-2023-PRE-Ofício Circular,2023-11-09,187-2023-PRE-Ofício Circular,"Informamos que, a partir de 13/11/2023, inclus...",Alteração na Regra de Cadastro Automático de V...,https://www.b3.com.br/data/files/73/E7/C5/27/2...,2023-11-09|187-2023-PRE-Ofício Circular.pdf,Este documento produz efeitos a partir da data...,9 de novembro de 2023 187/2023-PRE OFÍCIO CIRC...
107-2023-VNC-Comunicado Externo,2023-11-09,107-2023-VNC-Comunicado Externo,"A B3 informa que, conforme Comunicado Externo ...",Lançamento das sessões de Renda Fixa Trademate,https://www.b3.com.br/data/files/78/67/00/8C/C...,2023-11-09|107-2023-VNC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...,9 de novembro de 2023 107/2023-VNC COMUNICADO ...
058-2023-VPC-Comunicado Externo,2023-11-09,058-2023-VPC-Comunicado Externo,"Informamos que, em 27/10/2023, a Receita Feder...",Imposto de Renda da Pessoa Física (IRPF) – Cál...,https://www.b3.com.br/data/files/43/00/01/01/9...,2023-11-09|058-2023-VPC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...,9 de novembro de 2023 058/2023-VPC COMUNICADO ...
059-2023-VPC-Comunicado Externo,2023-11-09,059-2023-VPC-Comunicado Externo,"A B3 informa que, os dados indicados neste Com...",Atualização cadastral de companhias emissoras ...,https://www.b3.com.br/data/files/31/60/A7/FF/A...,2023-11-09|059-2023-VPC-Comunicado Externo.pdf,Este documento produz efeitos a partir da data...,9 de novembro de 2023 059/2023-VPC COMUNICADO ...
186-2023-PRE-Ofício Circular,2023-11-09,186-2023-PRE-Ofício Circular,"Informamos que, a partir de 27/11/2023, serão ...",Novas Regras e Parâmetros para Procedimentos E...,https://www.b3.com.br/data/files/0F/44/94/03/7...,2023-11-09|186-2023-PRE-Ofício Circular.pdf,Este documento produz efeitos a partir da data...,9 de novembro de 2023 186/2023-PRE OFÍCIO CIRC...


In [10]:
docs_to_merge = docs_df[["published_abstract", "published_subject", "pypdfium_extraction", "cleaned_extraction"]]

In [11]:
merged_df = (
    data_df
    .merge(docs_to_merge, left_index=True, right_index=True, how="left")
    .head()
)

In [12]:
merged_df.head()

,published_date,url,file_name,responsible,snippet_1,snippet_2,snippet_3,snippet_4,snippet_5,length,published_abstract,published_subject,pypdfium_extraction,cleaned_extraction
published_title,,,,,,,,,,,,,,
187-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/73/E7/C5/27/2...,2023-11-09|187-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 13/11/2023, inclusive, será altera...",A regra de cadastro automático de vencimentos ...,os demais termos do Contrato Futuro de Cupom d...,A regra vigente de cadastro automático e cadas...,Esclarecimentos adicionais poderão ser obtidos...,487.0,"Informamos que, a partir de 13/11/2023, inclus...",Alteração na Regra de Cadastro Automático de V...,Este documento produz efeitos a partir da data...,9 de novembro de 2023 187/2023-PRE OFÍCIO CIRC...
107-2023-VNC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/78/67/00/8C/C...,2023-11-09|107-2023-VNC-Comunicado Externo.pdf,Gabriel,a data para a entrada do serviço em ambiente d...,"em 11/11/2023 entre 10h30 e 12h30, haverá exce...","possibilidade de realizar, no dia supracitado,...",Esclarecimentos adicionais poderão ser obtidos...,NaN,393.0,"A B3 informa que, conforme Comunicado Externo ...",Lançamento das sessões de Renda Fixa Trademate,Este documento produz efeitos a partir da data...,9 de novembro de 2023 107/2023-VNC COMUNICADO ...
058-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/43/00/01/01/9...,2023-11-09|058-2023-VPC-Comunicado Externo.pdf,Gabriel,"em 27/10/2023, a Receita Federal do Brasil (RF...",prevê a obrigatoriedade do envio das informaçõ...,A partir do aprendizado gerado pelas fases ant...,NaN,NaN,402.0,"Informamos que, em 27/10/2023, a Receita Feder...",Imposto de Renda da Pessoa Física (IRPF) – Cál...,Este documento produz efeitos a partir da data...,9 de novembro de 2023 058/2023-VPC COMUNICADO ...
059-2023-VPC-Comunicado Externo,2023-11-09,https://www.b3.com.br/data/files/31/60/A7/FF/A...,2023-11-09|059-2023-VPC-Comunicado Externo.pdf,Gabriel,"A B3 informa que, no dia 13/11/2023, atualizar...",Registro emissor CVM (Sim ou Não); e Fase (Ope...,A marcação de EGEM ou EFRF será retirada,A atualização em referência será realizada par...,Esclarecimentos adicionais poderão ser obtidos...,501.0,"A B3 informa que, os dados indicados neste Com...",Atualização cadastral de companhias emissoras ...,Este documento produz efeitos a partir da data...,9 de novembro de 2023 059/2023-VPC COMUNICADO ...
186-2023-PRE-Ofício Circular,2023-11-09,https://www.b3.com.br/data/files/0F/44/94/03/7...,2023-11-09|186-2023-PRE-Ofício Circular.pdf,Gabriel,"a partir de 27/11/2023, serão implementados no...",Quantidade média negociada,Quantidade em relação ao capital social do emi...,Negociabilidade,Padronização de leilões de opções,269.0,"Informamos que, a partir de 27/11/2023, serão ...",Novas Regras e Parâmetros para Procedimentos E...,Este documento produz efeitos a partir da data...,9 de novembro de 2023 186/2023-PRE OFÍCIO CIRC...


In [27]:
token_name = 'unicamp-dl/ptt5-base-portuguese-vocab'
model_name = 'phpaiola/ptt5-base-summ-xlsum'
tokenizer = T5Tokenizer.from_pretrained(token_name )
model_pt = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_single_t5_summary(text):
    inputs = tokenizer.encode(text, max_length=512, truncation=True, return_tensors='pt')
    summary_ids = model_pt.generate(inputs, max_length=256, min_length=32, num_beams=5, no_repeat_ngram_size=3, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0])
    return summary

In [29]:
text = merged_df[["cleaned_extraction"]].iloc[0, :][0]
text

/tmp/ipykernel_208559/3274754811.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = merged_df[["cleaned_extraction"]].iloc[0, :][0]


'9 de novembro de 2023 187/2023-PRE OFÍCIO CIRCULAR Participantes do Listado B3  Ref.: Alteração na Regra de Cadastro Automático de Vencimentos do  Futuro de Cupom de IPCA (DAP) Informamos que, a partir de 13/11/2023, inclusive, será alterada a regra para  cadastro automático de vencimentos do Contrato Futuro de Cupom de IPCA  (DAP).  A regra de cadastro automático de vencimentos foi revista, visando ampliar a  abertura de novos vencimentos curtos de DAP, de 3 primeiros meses para 6,  conforme tabela abaixo Regra até 10/11/2023 Regras do  contrato Cadastro automático Cadastro pré-aprovado Restrições Todos os meses 3 primeiros meses. 1º vencimento janeiro (não  contemplado nos 3 primeiros  meses). Até 15 anos: coincidente com  vencimento de NTN-B (maio e  agosto). Até 15 anos • Coincidente com pagamento de  juros de NTN-B (fevereiro, maio,  agosto e novembro). • Início de semestre. -Regra a partir de 13/11/2023 Regras do  contrato Cadastro automático Cadastro pré-aprovado Restrições Tod

In [30]:
print(generate_single_t5_summary(text))

<pad> A partir de 13 de novembro de 2023, o Banco Central (BC) vai mudar a regra de cadastro automático de vencimentos do Futuro de Cupom de Créditos (IPCA).</s>


In [31]:
text = merged_df[["cleaned_extraction"]].iloc[1, :][0]
text

/tmp/ipykernel_208559/3128727489.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = merged_df[["cleaned_extraction"]].iloc[1, :][0]


'9 de novembro de 2023 107/2023-VNC COMUNICADO EXTERNO Participantes do Balcão B3  Ref.: Lançamento das sessões de Renda Fixa Trademate A B3 informa que, conforme Comunicado Externo 104/2023-VNC, desde  08/11/2023, após o encerramento da negociação, estão disponíveis em  ambiente de certificação, as sessões de Renda Fixa Trademate: Market Data,  Order Entry e Drop Copy.  Com o intuito de prover tempo adicional para certificação dos participantes, a data para a entrada do serviço em ambiente de produção será postergada para  o dia 27/11/2023. Reforçamos que a certificação é obrigatória e deverá ser agendada previamente com a Superintendência de Listados – Certificação pelo telefone (11) 2565-5023 ou e-mail tradingcertification@b3.com.br.  Esclarecemos que, após a aprovação dos participantes em ambiente de  certificação, será concedido o acesso às sessões em produção.  A versão do Trademate a ser disponibilizada em ambiente de certificação  contemplará, além das Sessões FIX de Renda Fixa

In [32]:
generate_single_t5_summary(text)

'<pad> A Superintendência de Listados (STN, na sigla em inglês) do Serviço Nacional de Certificação (SNI) anunciou nesta sexta-feira que o serviço de certificação de Renda Fixa será disponibilizado em ambiente de certificação para os participantes do Balcão B3.</s>'

In [33]:
text = merged_df[["cleaned_extraction"]].iloc[2, :][0]
text

/tmp/ipykernel_208559/3645145852.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = merged_df[["cleaned_extraction"]].iloc[2, :][0]


'9 de novembro de 2023 058/2023-VPC COMUNICADO EXTERNO Participantes do Listado B3  Ref.: Imposto de Renda da Pessoa Física (IRPF) – Cálculo Darf e Declaração  Anual – ReVar Informamos que, em 27/10/2023, a Receita Federal do Brasil (RFB) publicou a Instrução Normativa RFB 2.164 (IN) que, dentre outros pontos, prevê a  obrigatoriedade do envio das informações de corretagem pelos participantes para a B3 – depositária central. Essa IN compreende mais uma das importantes etapas de implementação da calculadora ReVar, uma iniciativa conjunta da RFB e B3, para apuração de Imposto  de Renda de Pessoas Físicas (IRPF) em operações de renda variável. A ReVar trará aos investidores pessoas físicas segurança e transparência para se  manter em dia com as obrigações tributárias relacionadas às operações de renda  variável, facilitando sua jornada de investimentos. Em 24/10/2023, a RFB, com a participação da B3, apresentou ao mercado os  detalhes da IN, o cronograma de implementação da calculadora Re

In [34]:
generate_single_t5_summary(text)

'<pad> A Agência do Investidor da Bolsa de Valores de São Paulo (Sinacor) divulgou nesta sexta-feira o cronograma de implementação da calculadora ReVar, que prevê a obrigatoriedade do envio das informações de corretagem dos investidores em operações de renda variável.</s>'

In [35]:
text = merged_df[["cleaned_extraction"]].iloc[3, :][0]
text

/tmp/ipykernel_208559/3557193404.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = merged_df[["cleaned_extraction"]].iloc[3, :][0]


'9 de novembro de 2023 059/2023-VPC COMUNICADO EXTERNO Participantes do Balcão B3  Ref.: Atualização cadastral de companhias emissoras de valores mobiliários no  Balcão B3 A B3 informa que, no dia 13/11/2023, atualizará os seguintes dados relacionados  ao cadastro das companhias emissoras de valores mobiliários registradas na  Comissão de Valores Mobiliários (CVM) com status de registro ativo:  • Registro emissor CVM (Sim ou Não); e • Fase (Operacional ou pré-operacional). Para essa atualização, serão utilizadas as informações cadastrais das companhias  abertas publicadas no site da CVM, disponibilizadas em  https://dados.cvm.gov.br/dataset/cia_aberta-cad.  A marcação de EGEM ou EFRF será retirada, conforme critérios dos artigos 38 e 38-A da Resolução CVM 80, para regularização do cadastro das companhias  emissoras não registradas na CVM e cadastradas na B3 como Emissores com  Grande Exposição ao Mercado e Emissores Frequentes de Renda Fixa.  A atualização em referência será realizada 

In [36]:
generate_single_t5_summary(text)

'<pad> Empresas emissoras de valores mobiliários no Balcão B3, em São Paulo (SP), serão obrigadas a atualizar seus cadastros e informações cadastrais para a Comissão de Valores Mobiliários (CVM).</s>'

In [37]:
text = merged_df[["cleaned_extraction"]].iloc[4, :][0]
text

/tmp/ipykernel_208559/2560065442.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = merged_df[["cleaned_extraction"]].iloc[4, :][0]


'9 de novembro de 2023 186/2023-PRE OFÍCIO CIRCULAR Participantes do Listado B3  Ref.: Novas Regras e Parâmetros para Procedimentos Especiais de  Negociação Informamos que, a partir de 27/11/2023, serão implementados novos  parâmetros simplificados para procedimentos especiais de negociação, conforme  descrito a seguir, em virtude da publicação da Resolução CVM 135, que revogou  a Instrução CVM 168, e estabeleceu que as entidades administradoras de  mercado organizado devem desenvolver e manter regras de organização e  funcionamento de seus ambientes e sistemas de negociação. Quantidade média negociada Será estabelecida simplificação do parâmetro de leilão, que equivalerá a 20 vezes  a média nacional negociada, acompanhada de redução no tempo de leilão para  5 minutos. Quantidade em relação ao capital social do emissor Com o objetivo de abranger tanto as ações ordinárias quanto as preferenciais em  relação ao capital social do emissor, os valores serão consolidados em apenas um  parâme

In [38]:
generate_single_t5_summary(text)

'<pad> A Comissão de Valores Múltiplos (CVM) aprovou nesta quarta-feira uma nova regra para o leilão de ativos, que será realizado a partir de 27 de novembro.</s>'